# Introdução


A Corp Solutions, uma empresa de consultoria empresarial, está no centro das atenções por sua venda de softwares inovadores voltados para o setor de Recursos Humanos. Recentemente, a empresa recebeu um aporte significativo de uma grande corporação dos Estados Unidos, impulsionando sua expansão e reforçando seu compromisso em investir em iniciativas de diversidadra o sucesso.

## Contexto Histórico

Nos últimos anos, temos testemunhado uma mudança significativa no mundo organizacional em direção à valorização da diversidade e inclusão. À medida que a sociedade evolui, as empresas reconhecem cada vez mais os benefícios de uma força de trabalho diversificada, incluindo maior inovação, criatividade e desempenho financeiro. Além disso, movimentos sociais e pressões externas têm levado as empresas a adotarem práticas mais inclusivas e equitativas.

## O Desafio da Corp Solutions

A Corp Solutions acredita que a diversidade não é apenas uma questão de representatividade, mas sim uma oportunidade de impulsionar a inovação, a criatividade e o sucesso dos negócios. Reconhecendo a importância de um ambiente de trabalho inclusivo, a empresa está empenhada em criar uma cultura onde todas as vozes sejam ouvidas, valorizadas e respeitadas.
Com o recente investimento, a Corp Solutions planeja expandir sua equipe, dobrando o número de colaboradores até o final de 2025, atingindo o impressionante total de 850 pessoas dentro da corporação. Essa expansão não se trata apenas de números; é uma oportunidade para fortalecer a diversidade e a inclusão em todos os níveis da organização.

## AdaHack: Diversidade e Inclusão

Neste Case do AdaHack sobre Diversidade e Inclusão, exploraremos a jornada da Corp Solutions em busca de um ambiente mais diversificado e acolhedor. Vamos analisar como a empresa está enfrentando esse desafio, quais estratégias estão sendo implementadas e como os dados podem nos guiar nessa jornada.

# Documentação técnica

## Importação das bibliotecas

In [ ]:
# Importando as bibliotecas numpy e pandas, importante para tratamento dos dados importados
import numpy as np
import pandas as pd

# Importando a biblioteca pyplot e seaborn, importante para visualização dos dados importados
import matplotlib.pyplot as plt
import seaborn as sns

# Importando a biblioteca requests, para requisitar a fonte dos dados (Github)
import requests

# Importando bibliotecas io e ftfy, para tratamento de codificação de caracteres truncados (utf-8)
from io import StringIO
import ftfy

# Importando a biblioteca scipy, para auxílio de análises estatísticas
from scipy.stats import f_oneway

#
from IPython.display import display

## Carregamento dos dados

In [ ]:
# Carregando dados diretamento do github e corrigindo problemas de codificação
url = requests.get('https://raw.githubusercontent.com/Ada-Empregabilidade/adahack-2024-dados/main/base_dados/base_de_dados.csv').content.decode('utf-8')
linhas = url.split('\n')
lista_linhas = [ftfy.fix_text(linha) for linha in linhas]
data = StringIO('\n'.join(lista_linhas))
df = pd.read_csv(data)

_Optamos por utilizar as informaçoes diretamente da fonte de dados, ao invés de baixa-las e adiciona-las manualmente. O arquivo csv fornecido contém problemas de codificação e também optamos por fazer o tratamento dentro do notebook._

## Exploração inicial dos dados

#### Primeiro contato com os dados

In [ ]:
# Comando head para exibir as 10 primeiras linhas do dataset
df.head(10)

_Aqui já encontramos algumas anomalias, que serão tratadas ao longo do processo de limpeza, tais como:_
1. _Coluna 'id' e 'id.1' duplicadas._
2. _Coluna 'Idade' com valores negativos._
3. _'Endereço' estadunidense com uma quebra de linha não reconhecida_
4. _'Formação' possui valores nulos._
5. _Coluna 'Tempo de casa' com valores negativos_
6. _Coluna 'Senioridade' com diferentes grafias para um mesmo cargo_

#### Compreendendo as variáveis qualitativas

In [ ]:
#Valores únicos para a variável 'Raça'
df['Raça'].unique()

In [ ]:
#Valores únicos para a variável 'Genero'
df['Genero'].unique()

In [ ]:
#Valores únicos para a variável 'Formação'
df['Formação'].unique()

In [ ]:
#Valores únicos para a variável 'Estado'
df['Estado'].unique()

In [ ]:
#Valores únicos para a variável 'Departamento'
df['Departamento'].unique()

In [ ]:
#Valores únicos para a variável 'Senioridade'
df['Senioridade'].unique()

_Podemos compreender os diferentes tipos de variáveis, e como estão distribuidas ao longo do dataset_

#### Tamanho da base de dados

In [ ]:
#Entendendo o tamanho da base de dados
print(f'O presente dataset posssui {df.shape[0]} linhas e {df.shape[1]} colunas. ')

In [ ]:
# Listando colunas
df.columns

#### Estatística Descritiva

In [ ]:
# Panorama estatístico das colunas numéricas do dataset
df.describe()

In [ ]:
# Panorama estatístico das colunas não-numéricas do dataset
df_nao_numerico = df.select_dtypes(exclude='number')
df_nao_numerico.describe()

_O nome Francisco da Silva aparentemente está duplicado. Mas por ser uma combinação de nome e sobrenome comuns, vamos deixa-lo._

#### Tipos de variáveis

In [ ]:
# Verificando tipos de variáveis por cada coluna.
tipos_de_dados = df.info()
display(tipos_de_dados)

_Acima percebemos que os tipos de dados estão praticamente conformes, exceto pela coluna 'Idade', que não precisa ser do tipo float e deve ser um número inteiro. O mesmo acontece com a coluna 'Tempo de casa'._

## Limpeza e transformação dos dados

#### Lidando com dados faltantes

In [ ]:
# Relacionando dados faltantes
df.isna().sum()

_Considerando que esses dados faltantes, como de atributos diferentes temos: 56+44+200 = 300. Como temos um total de 10000 observações é razoalvel eliminar esses dados, ao invés de preencher com valores tais como média, mediana, etc.. Representa eliminar 300/10000, ou seja, 3% dos dados._

In [ ]:
# Removendo dados faltantes
df = df.dropna(axis=0, how='any', inplace=False)

#### Tipos de variáveis

In [ ]:
# Verificando tipos de variáveis das colunas
df.info()

In [ ]:
#Verificando se realmente existem números decimais nas colunas 'Idade' e 'Tempo de casa'
df_decimais = df.loc[(df['Idade'] % 1 != 0) & (df['Tempo de casa'] % 1 != 0)]
df_decimais.info()

In [ ]:
# Convertendo as colunas Idade e Tempo de Casa para (INT)
df['Idade'] = df['Idade'].astype(int)
df['Tempo de casa'] = df['Tempo de casa'].astype(int)

In [ ]:
# Conferindo resultados
df.info()

#### Removendo linhas duplicadas

In [ ]:
# Remover linhas duplicadas
df = df.drop_duplicates()

In [ ]:
# Identificação e exibição de registros duplicados considerando a coluna 'Nome'

registros_duplicados_nome = df[df.duplicated(subset=['Nome'], keep=False)]
registros_duplicados = registros_duplicados_nome.sort_values(by='Nome')
display(registros_duplicados_nome)

In [ ]:
# Demonstra o número de registros duplicados da coluna 'Nome'.

len(registros_duplicados_nome)

#### Removendo colunas desnecessárias

In [ ]:
#Remover colunas desnecessárias
df = df.drop(columns=['id', 'id.1', 'Nome', 'Endereço'])

#### Removendo Outliers

In [ ]:
# Análise com gráfico Boxplot

numerical_columns = df.select_dtypes(include='number').columns

plt.figure(figsize=(10, 6))
for column in numerical_columns:
    df.boxplot(column=column)
    plt.title(f'Boxplot for {column}')
    plt.show()

In [ ]:
# Removendo Outliers na coluna 'Idade':

Q1_idade = np.percentile(df['Idade'], 25)
Q3_idade = np.percentile(df['Idade'], 75)

IQR_idade = Q3_idade - Q1_idade

idade_minima = Q1_idade - 1.5 * IQR_idade
idade_maxima = Q3_idade + 1.5 * IQR_idade

df = df[(df['Idade'] >= idade_minima) & (df['Idade'] <= idade_maxima)]

In [ ]:
# Removendo Outliers na coluna 'Tempo de casa':

Q1_tempo = np.percentile(df['Tempo de casa'], 25)
Q3_tempo = np.percentile(df['Tempo de casa'], 75)

IQR_tempo = Q3_tempo - Q1_tempo

tempo_minima = Q1_tempo - 1.5 * IQR_tempo
tempo_maxima = Q3_tempo + 1.5 * IQR_tempo

df = df[(df['Tempo de casa'] >= tempo_minima) & (df['Tempo de casa'] <= tempo_maxima)]

#### Removendo dados inválidos

##### Tempo de casa negativo

In [ ]:
# Encontrando tempos de casa negativos
tempo_casa_negativo = df.loc[df['Tempo de casa'] < 0]
display(tempo_casa_negativo)

_Analisando a coluna 'Tempo de casa' encontramos 96 observações com valor negativo. O que não parece fazer sentido. Como repesentam 96/9701 = 0.01, ou seja, 1% é razoável eliminar essas linhas._

In [ ]:
# Removendo pessoas com mais tempo negativos
df.drop(tempo_casa_negativo.index, inplace=True)

##### Tempo de casa zerado

In [ ]:
# Encontrando tempos de casa zerados
tempo_casa_zero = df.loc[df['Tempo de casa'] == 0]
display(tempo_casa_zero)

_Os valores zerados para a variável 'Tempo de casa', não serão excluídas, porque entendemos não serem inválidas, apenas que os funcionários não alcançaram 1 (um) ano. Pois assumimos que esse atributo é medido em ano._

##### Menores de idade

In [ ]:
# Encontrando Idades inválidas (menores de idade)
menor_idade = df.loc[df['Idade'] < 18]
display(menor_idade)

In [ ]:
# Removendo menores de idade
df = df.drop(df[df['Idade'] < 18].index)

##### Pessoas com mais experiência que idade

In [ ]:
# Encontrando Idades maiores que o Tempo de casa
tempo_casa_idade = df[df['Tempo de casa'] > df['Idade']]
display(tempo_casa_idade)

In [ ]:
# Removendo pessoas com mais tempo de casa do que idade
df.drop(tempo_casa_idade.index, inplace=True)

##### Menores de idade na contratação

In [ ]:
tempo_casa_menor_idade = df[(df['Idade'] - df['Tempo de casa']) < 18]
display(tempo_casa_menor_idade)

_Se subtrairmos o tempo de casa da idade do colaborador, chegaremos a idade na contratação. O que nos retorna 2103 registros com pessoas menores de 18 anos no momento da contratação (excluindo a hipótese de que possam ter sido admitidos Menores Aprendizes, devido sua ausência na coluna Senioridade). Constatamos números absurdos, como contratações aos 5 anos de idade._

In [ ]:
# Removendo contratação de menores de idade
df.drop(tempo_casa_menor_idade.index, inplace=True)

#### Corrigindo grafias

In [ ]:
df['Senioridade'] = df['Senioridade'].replace('gerente', 'Gerente')

In [ ]:
df['Genero'] = df['Genero'].replace({'Fem' : 'Feminino', 'Masc' : 'Masculino'})

In [ ]:
df['Raça'] = df['Raça'].replace({'preto' : 'Preto', 'pardo' : 'Pardo', 'branco' : 'Branco', 'indígena' : 'Indígena', 'amarelo' : 'Amarelo'})

In [ ]:
df = df.rename(columns={'Genero': 'Gênero', 'Tempo de casa' : 'Tempo de Empresa'})

#### Coluna diversidade

In [ ]:
# Criando coluna Diversidade para facilitar a visualização das diversidades presentes
df['Diversidade'] = df ['Raça'] != 'branco'

#### Organização

In [ ]:
# Alterando a ordem das colunas para uma melhor visualização
nova_ordem_colunas = ['Gênero', 'Idade', 'Raça', 'Diversidade', 'Formação', 'Estado', 'Tempo de Empresa', 'Departamento', 'Senioridade']
df = df.reindex(columns=nova_ordem_colunas)

In [ ]:
#Conferindo resultados das transformações
df.head(10)

#### Exportação

In [ ]:
# Exportação dos dados limpos e organizados para importação no Power BI
df.to_csv('base_de_dados_tratada.csv')

## Análise estatística

#### Descrição dos dados

In [ ]:
df[['Idade', 'Tempo de Empresa']].describe()

_A idade média é de aproximadamento 30,92 anos. O desvio padrão que mede a dispersão dos dados em relação a média de 8,66 anos.
Os quartis que representam a distribuição indicam que 25% dos funcionários têm 24 anos ou menos, 50% dos funcionários têm 30 anos ou menos, e 75% dos funcionários têm 37 anos ou menos._

_O tempo de empresa média é de aproximadamento 5,56 anos. O desvio padrão que mede a dispersão dos dados em relação a média de 2,89 anos.
Os quartis que representam a distribuição indicam que 25% dos funcionários têm 4 anos de casa ou menos, 50% dos funcionários têm 6 anos de casa ou menos, e 75% dos funcionários têm 8 anos de casa ou menos._

#### Correlação dos dados

In [ ]:
correlacao = df[['Idade', 'Tempo de Empresa']].corr()
print(correlacao)


_A matriz de correlação indica que as Colunas Idade e Tempo de casa pouco influenciam uma com a outra._

#### P-valor

In [ ]:
departamentos = [df[df['Departamento'] == departamento]['Idade'] for departamento in df['Departamento'].unique()]
resultado = f_oneway(*departamentos)

print(f"F: {resultado.statistic:.2f}")
print(f"P-value: {resultado.pvalue:.2f}")


_Não podemos concluir que há diferenças significativas nas idades dos funcionários entre os diferentes departamentos._

## Análise e exploração dos dados

### Como os colaboradores estão distribuídos

#### Por Gênero

In [ ]:
contagem_genero = df['Gênero'].value_counts()
sns.set(style="whitegrid")

plt.figure(figsize=(8, 6))
barras = sns.barplot(x=contagem_genero.index, y=contagem_genero.values, palette="pastel", hue=contagem_genero.index, dodge=False, legend=False)

plt.title('Distribuição dos Colaboradores x Gênero')
plt.xlabel('Gênero')
plt.ylabel('Colaboradores')

for barra in barras.patches:
    barras.annotate(format(barra.get_height(), '.0f'), 
                   (barra.get_x() + barra.get_width() / 2, 
                    barra.get_height()), 
                   ha='center', va='center',
                   size=12, xytext=(0, 5),
                   textcoords='offset points')

total = df.shape[0]
for barra in barras.patches:
    barras.annotate(f'{barra.get_height() / total:.2%}', 
                   (barra.get_x() + barra.get_width() / 2, 
                    barra.get_height() / 2), 
                   ha='center', va='center',
                   size=20, xytext=(0, 0),
                   textcoords='offset points', color='white')
plt.show()

_Os dados apontam um desequilíbrio de gênero, buscando na referência uma meta. Assim, uma primeira análise se faz necessária por gênero.



*Com base na visualização da distribuição por gênero, observamos que a maioria dos registros está associada ao gênero masculino (7194 registros), representando aproximadamente 74,9% do total, enquanto o gênero feminino corresponde a cerca de 25,1% (2411 registros). Essa disparidade sugere uma possível desproporção na representação de gênero nos dados analisados. Tal análise inicial pode fornecer insights valiosos para compreender melhor a dinâmica dos dados e pode ser o ponto de partida para investigações mais aprofundadas sobre questões de equidade de gênero ou viés nos conjuntos de dados analisados.*
_

#### Por Idade

In [ ]:
sns.set(style="whitegrid")

plt.figure(figsize=(8, 6))
histograma = sns.histplot(data=df, x='Idade', bins=10, kde=True)

plt.title('Distribuição de Idade')
plt.xlabel('Idade')
plt.ylabel('Número de Colaboradores')

plt.show()

_Apesar de bem distribuída, a Corp solutios apresenta em sua maioria colaboradores na faixa de 30-40 anos._

#### Por Raça

In [ ]:
contagem_raca = df['Raça'].value_counts()
sns.set(style="whitegrid")

plt.figure(figsize=(8, 6))
barras = sns.barplot(x=contagem_raca.index, y=contagem_raca.values, hue=contagem_raca.index, palette="pastel")

plt.title('Distribuição dos Colaboradores por Raça')
plt.xlabel('Raça')
plt.ylabel('Número de Colaboradores')

for barra in barras.patches:
    barras.annotate(format(barra.get_height(), '.0f'), 
                   (barra.get_x() + barra.get_width() / 2, 
                    barra.get_height()), 
                   ha='center', va='center',
                   size=12, xytext=(0, 5),
                   textcoords='offset points')

plt.show()

_Consultar referência para entender se faz sentido essa distribuição de raça. Contudo, chama atenção a frequência de pardos, nitidamente maior que pretos._

#### Por Escolaridade

In [ ]:
contagem_escolaridade = df['Formação'].value_counts()
sns.set(style="whitegrid")

plt.figure(figsize=(8, 6))
barras = sns.barplot(x=contagem_escolaridade.index, y=contagem_escolaridade.values, hue=contagem_escolaridade.index, palette="pastel")

plt.title('Distribuição dos Colaboradores por Nível de Escolaridade')
plt.xlabel('Nível de Escolaridade')
plt.ylabel('Número de Colaboradores')

for barra in barras.patches:
    barras.annotate(format(barra.get_height(), '.0f'), 
                   (barra.get_x() + barra.get_width() / 2, 
                    barra.get_height()), 
                   ha='center', va='center',
                   size=12, xytext=(0, 5),
                   textcoords='offset points')

plt.show()

_Chama atenção a frequência de ensino superior, consideravelmente maior que os outros casos. Existe uma tendência da empresa em contratar pessoas com formação de ensino superior. Indica uma possibilidade de subemprego, está de acordo com as referências do estudo._

#### Por Região (Nordeste, Sul, etc.)

In [ ]:
regioes = {
    'Acre': 'Norte', 'Alagoas': 'Nordeste', 'Amapá': 'Norte', 'Amazonas': 'Norte', 'Bahia': 'Nordeste', 'Ceará': 'Nordeste',
    'Distrito Federal': 'Centro-Oeste', 'Espírito Santo': 'Sudeste', 'Goiás': 'Centro-Oeste', 'Maranhão': 'Nordeste', 'Mato Grosso': 'Centro-Oeste',
    'Mato Grosso do Sul': 'Centro-Oeste', 'Minas Gerais': 'Sudeste', 'Pará': 'Norte', 'Paraíba': 'Nordeste', 'Paraná': 'Sul', 'Pernambuco': 'Nordeste',
    'Piauí': 'Nordeste', 'Rio de Janeiro': 'Sudeste', 'Rio Grande do Norte': 'Nordeste', 'Rio Grande do Sul': 'Sul', 'Rondônia': 'Norte', 'Roraima': 'Norte', 
    'Santa Catarina': 'Sul', 'São Paulo': 'Sudeste', 'Sergipe': 'Nordeste', 'Tocantins': 'Norte'
}

df['Região'] = df['Estado'].map(regioes)

contagem_regiao = df['Região'].value_counts()


sns.set(style="whitegrid")
plt.figure(figsize=(8, 6))
sns.barplot(x=contagem_regiao.index, y=contagem_regiao.values, hue=contagem_regiao.index, palette="pastel")
plt.title('Distribuição dos Colaboradores por Região do Brasil')
plt.xlabel('Região')
plt.ylabel('Número de Colaboradores')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

_A distribuição regional indica que realmente a Corp Solutions é uma empresa oferece jornada flexíveis e modelos de trabalho remotos, com a distribuição pulverizada pelas regiões do Brasil.
Observamos uma discreta vantagem no Sudeste, porém justificável como sendo o centro comercial do país._

#### Por Estado (UF)

In [ ]:
uf = {
        'Acre': 'AC', 'Alagoas': 'AL', 'Amapá': 'AP', 'Amazonas': 'AM', 'Bahia': 'BA',
        'Ceará': 'CE', 'Distrito Federal': 'DF', 'Espírito Santo': 'ES', 'Goiás': 'GO',
        'Maranhão': 'MA', 'Mato Grosso': 'MT', 'Mato Grosso do Sul': 'MS', 'Minas Gerais': 'MG',
        'Pará': 'PA', 'Paraíba': 'PB', 'Paraná': 'PR', 'Pernambuco': 'PE', 'Piauí': 'PI',
        'Rio de Janeiro': 'RJ', 'Rio Grande do Norte': 'RN', 'Rio Grande do Sul': 'RS',
        'Rondônia': 'RO', 'Roraima': 'RR', 'Santa Catarina': 'SC', 'São Paulo': 'SP',
        'Sergipe': 'SE', 'Tocantins': 'TO'
}
    
df['UF'] = df['Estado'].map(uf)
contagem_estado = df['UF'].value_counts()

sns.set(style="whitegrid")
plt.figure(figsize=(8, 6))
sns.barplot(x=contagem_estado.index, y=contagem_estado.values, hue=contagem_estado.index, palette="pastel")
plt.title('Distribuição dos Colaboradores por Estado do Brasil')
plt.xlabel('Estado')
plt.ylabel('Número de Colaboradores')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

_Os gráficos mostram uma tendência em colaboradores da região sudeste e do estado de SP, RJ, MG, PR, SC e RS. A estratégia de equilibrar por região pode ser difícil dependendo da sua localização.
Lembrando que esquilibrar por região é algo relavante, pois uma mistura de diferentes regiões traz uma riqueza cultural ao dia a dia de trabalho._

#### Por Tempo de empresa

In [ ]:
plt.figure(figsize=(8, 6))
plt.hist(df['Tempo de Empresa'], bins=18, color='skyblue', edgecolor='black')
plt.title('Histograma de tempo de empresa')
plt.xlabel('Tempo de empresa')
plt.ylabel('Frequência')
plt.grid(True)
plt.show()

_Os dados mostram frequência muito elevada de pessoas com 4 a 7,5 anos de trabalho e uma moderada frequência de novos colaboradores, assim indica que a estratégia interessante é trabalhar a diversidade com a projeção de contratação de novos colaboradores._

#### Por Departamento

In [ ]:
department_counts = df['Departamento'].value_counts()

plt.figure(figsize=(10, 6))
department_counts.plot(kind='bar', color='skyblue', edgecolor='black')  
plt.title('Histograma de departamento')
plt.xlabel('Departamento')
plt.ylabel('Frequência')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y')
plt.show()

_A distribuição da equipe se apresenta como nivelada entre os departamentos._

#### Por Senioridade

In [ ]:
senioridade_counts = df['Senioridade'].value_counts()

plt.figure(figsize=(8, 6))
senioridade_counts.plot(kind='bar', color='skyblue', edgecolor='black')  
plt.title('Histograma de Senioridade')
plt.xlabel('Senioridade')
plt.ylabel('Frequência')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y')
plt.show()

_A senioridade apresenta dados naturais ao desenvolvimento de pessoas, visto que pessoas com menor senioridade apresentam um gasto menor para a empresa. Assim, é natural ter um número menor de pessoas com alto nível de senioridade._

### Como as variáveis se relacionam?

#### Gênero e Raça

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x='Gênero', hue='Raça', data=df, palette='Set2')
plt.title('Distribuição de gênero por raça')
plt.xlabel('Gênero')
plt.ylabel('Contagem')
plt.legend(title='Raça')
plt.grid(axis='y')
plt.show()

_Os gráficos apresentam relações semelhantes, contudo é possível notar uma maior quantidade de homens pardos em relação às mulheres pardas, o que pode sugerir a necessidade de contração de mais mulheres pardas. Contudo, devido à apresetarem resultados semelhantes, não é possível estabelecer um viés de contratação._

#### Gênero e Formação

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x='Gênero', hue='Formação', data=df, palette='Set2')
plt.title('Distribuição de gênero por formação')
plt.xlabel('Gênero')
plt.ylabel('Contagem')
plt.legend(title='Formação')
plt.grid(axis='y')

plt.show()

*A análise da distribuição por formação, diferenciando por gênero, revela insights interessantes sobre a educação dos indivíduos no conjunto de dados. Ao considerar os diferentes níveis de formação, observamos que, em todos os níveis, há uma predominância de homens em relação às mulheres. Por exemplo, no nível de ensino superior, os homens superam as mulheres com uma diferença de aproximadamente 3 para 1. Essa disparidade se mantém em outros níveis de formação, como pós-graduação, mestrado, doutorado e ensino médio. Essa análise sugere disparidades de gênero na educação, destacando a importância de políticas e iniciativas para promover a igualdade de oportunidades educacionais para todos os gêneros.*

#### Gênero e Idade

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(hue='Gênero', y='Idade', data=df, palette='Set2', legend=False)
plt.title('Distribuição de idade por gênero.')
plt.xlabel('Gênero')
plt.ylabel('Idade')
plt.grid(axis='y')
plt.show()

_A distribuição entre idades não parece ser diferenciada conforme o gênero._

#### Gênero e Região

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x='Gênero', hue='Região', data=df, palette='Set2')
plt.title('Distribuição de gênero por região')
plt.xlabel('Gênero')
plt.ylabel('contagem')
plt.legend(title='Região')
plt.xticks(rotation=0)
plt.grid(axis='y')

plt.show()

_Não foram observadas conclusões ou uma evidente correlação entre o Gênero e Região._

#### Gênero e Tempo de empresa

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='Tempo de Empresa', hue='Gênero', palette='Set2', multiple='stack', edgecolor='black', bins=30)
plt.title('Distribuição de gênero por tempo de empresa')
plt.xlabel('Tempo de empresa')
plt.ylabel('Contagem')
plt.grid(axis='y')
plt.show()

_A análise do gráfico não apresenta alguma influência do gênero em relação ao tempo de empresa, sugerindo que ainda podem não ter sido implantadas políticas de diversidade. Ressalta-se que, para os colaboradores com maior tempo de empresa, diminui a quantidade de mulheres._

#### Gênero e Departamento

In [ ]:
plt.figure(figsize=(12, 6))  # Adjust the figure size if needed
sns.countplot(x='Departamento', hue='Gênero', data=df, palette='Set2')
plt.title('Distribuição de gênero por departamento')
plt.xlabel('Departamento')
plt.ylabel('Contagem')
plt.legend(title='Gênero')
plt.grid(axis='y')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better visibility
plt.show()

_A distribuição de vagas no departamento mantém o nivelamento, reforçando a necessidade de contratação de mais mulheres._

#### Gênero e Senioridade?

In [ ]:
plt.figure(figsize=(10, 6))  # Adjust the figure size if needed
sns.countplot(x='Gênero', hue='Senioridade', data=df, palette='Set2')
plt.title('Distribuição de gênero e Senioridade')
plt.xlabel('Gênero')
plt.ylabel('Contagem')
plt.legend(title='Senioridade')
plt.xticks(rotation=45)  # Rotate x-axis labels for better visibility
plt.grid(axis='y')
plt.show()

_Analisando esse gráfico, é possível notar que o gênero não influencia na hierarquia, destaque para o cargo de diretoria que não aparece nos gráficos do gênero feminino. O que sugere que é necessária uma política de aumentar a contratação de mulheres e que futuramente pode equilibrar esses cargos._

# Comunicação dos resultados

## Situação atual Corp Solutions

## Diversidade

(GPTW)
Os negros são 56% da população brasileira e movimentam R$ 1,7 tri por ano,
apesar de terem uma renda média R$ 1.200 menor que dos brancos.

3 de agosto: se todos recebessem um salário diário igual em um ano, a mulher
negra começaria a ser remunerada apenas nessa data.

Empresas mais diversas e melhores: as empresas premiadas do GPTW étnico-racial
também são mais diversas em relação a gênero, e apresentam rotatividade
voluntária 88% menor que a média no Brasil.

(MCKINSEY)

A diversidade de gênero está correlacionada tanto com a lucratividade como com a criação de valor. No conjunto de dados de 2017, encontramos uma correlação positiva entre diversidade de gênero nas equipes executivas e as duas medidas de performance financeira que adotamos: em todo o mundo, empresas do quartil superior em diversidade de gênero nas equipes executivas tinham uma probabilidade 21% maior de ter margem EBIT superior à de seus pares do quartil inferior; além disso, tinham uma probabilidade 27% maior de criar valor no longo prazo do que seus pares do quartil inferior, medido pela margem de lucro econômico (Quadro 2).




## Treinamento e Conscientização

## Novas contratações

Embora a empresa possa ter interesse em aumentar a representatividade, ela vai encontrar os profissionais com determinada características?
Implementar programas de recrutamento ativo direcionados a grupos minoritários sub-representados.
Estabelecer parcerias com organizações que apoiam candidatos de grupos minoritários para aumentar o alcance do recrutamento.
Soluções para formar profissionalizais na área (Potencia Tech é uma inspiração)
Garantir que os processos de seleção sejam imparciais e livres de preconceitos, utilizando técnicas de recrutamento cego quando apropriado.

## Inclusão: PCD

(GPTW)
Investir em pessoas com deficiência vale a pena
Monday, March 20, 2023 ©Great Place To Work® Institute, Inc. All Rights Reserved. 18
A lei de cotas existe desde 1991 no Brasil, mas cumpre-se apenas metade dela,
atualmente. 1
Multas: podem ser de até R$ 250 mil por pessoa com deficiência não contrataa.
2
Apenas 1% dos trabalhadores é uma pessoa com deficiência, mas 8,4% da
população brasileira acima de dois anos possui alguma deficncia.
3

## LGBTQIA+

(GPTW)

Retorno do índice LGBTQ100 (NYSE) = crescimento de 12,0%.
Retorno do índice S&P500 = crescimento de 7,8%.

O público LGBTQIA+ consome +25% que os heterossexuais;
64% das pessoas LGBTQIA+ ainda não se assumiram no trabalho.

Há o dobro de pessoas LGBTQIA+ nas empresas premiadas do que
nas não-premiadas, em todos os níveis.


## Plano de Ação

# Referências

Durante o desenvolvimento dessa análise buscamos embasar nossas conclusões em fontes externas:

[Indicadores para Diversidade e Inclusão: por que mensurar e quais métricas utilizar](https://treediversidade.com.br/indicadores-para-diversidade-e-inclusao/) (Tree, 2022)

[A diversidade como alavanca de performance](https://www.mckinsey.com/capabilities/people-and-organizational-performance/our-insights/delivering-through-diversity/pt-BR) (Mckinsey, 2017)

[Diversity, Equity & Inclusion - Benchmarking Survey](https://www.pwc.com.br/pt/estudos/servicos/consultoria-negocios/2023/pesquisa-global-de-diversidade-equidade-e-inclusao.html) (PwC, 2022)


[Ranking Diversidade](https://gptw.com.br/wp-content/uploads/2023/03/Rankings-Diversidade-2023-v.2.pdf)(GPTW, 2023